In [1]:
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_accuracy, get_error_measures, collect_stats, print_average_stats
from src.models import TwoLayerPerceptron, SynapticSNN, LeakySNN, DoubleLeakySNN
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Data

In [5]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)

In [6]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.75 * len(all_data)), round(0.15 * len(all_data)), round(0.1 * len(all_data))))

In [5]:
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

### Simple regressor

In [6]:
model1 = TwoLayerPerceptron(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model1)

TwoLayerPerceptron(
  (lin1): Linear(in_features=8, out_features=4000, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=4000, out_features=1, bias=True)
)


In [7]:
training_loop(model1, train_loader, valid_loader, device, num_epochs=50, lr=0.003, validation=True)

  2%|▏         | 1/50 [00:02<02:04,  2.54s/it]

MAE: 6.4051214022443475, MSE:79.3088850732351,
RMSE: 8.905553608464501, IA: 0.8785967859374101, MAPE: 42.524250340555184%


  4%|▍         | 2/50 [00:02<01:02,  1.30s/it]

MAE: 6.299213872755194, MSE:78.90802608016764,
RMSE: 8.883018973308998, IA: 0.8758917524999306, MAPE: 39.158839261370794%


  6%|▌         | 3/50 [00:03<00:41,  1.12it/s]

...

MAE: 6.004404685676443, MSE:69.19556829842571,
RMSE: 8.318387361648032, IA: 0.9096788244576383, MAPE: 38.21854005419703%


 98%|█████████▊| 49/50 [00:22<00:00,  2.38it/s]

MAE: 5.89169941168989, MSE:65.71339050484514,
RMSE: 8.106379617612609, IA: 0.9057495091706491, MAPE: 39.466435195674585%


100%|██████████| 50/50 [00:23<00:00,  2.13it/s]

MAE: 6.081988444907128, MSE:67.35857775491974,
RMSE: 8.207227166036025, IA: 0.9048248260874595, MAPE: 42.347686686804984%
Accuracy on validation dataset: 49.94%


In [8]:
print(f'Accuracy on test set: {get_accuracy(model1, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model1, test_loader, device=device, print_e=True)

Accuracy on test set: 51.23%
MAE: 6.275693345510721, MSE:76.93435649748643,
RMSE: 8.771223204176623, IA: 0.9031492589612802, MAPE: 43.58376729999973%


(6.275693345510721,
 76.93435649748643,
 8.771223204176623,
 0.9031492589612802,
 43.58376729999973)

### SNN with Leaky layer

In [9]:
model2 = LeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model2)

LeakySNN(
  (fc1): Linear(in_features=8, out_features=4000, bias=True)
  (lif): Leaky()
  (fc2): Linear(in_features=4000, out_features=1, bias=True)
)


In [10]:
training_loop(model2, train_loader, valid_loader, device, num_epochs=25, lr=0.0005, validation=True)

  4%|▍         | 1/25 [00:05<02:07,  5.33s/it]

MAE: 11.933502882305596, MSE:322.06663314020034,
RMSE: 17.946215008747675, IA: 0.40790693234759356, MAPE: 55.7255608113032%


  8%|▊         | 2/25 [00:10<01:57,  5.10s/it]

MAE: 9.149780340690834, MSE:170.43816668488344,
RMSE: 13.055196922485829, IA: 0.5967077899677402, MAPE: 55.389432357148706%


 12%|█▏        | 3/25 [00:15<01:55,  5.25s/it]

MAE: 7.715248570139009, MSE:115.35275177845116,
RMSE: 10.740239838032071, IA: 0.7777177457687532, MAPE: 49.156550814343106%


 16%|█▌        | 4/25 [00:21<01:54,  5.47s/it]

MAE: 7.0630519858674505, MSE:94.63635594455802,
RMSE: 9.728121912504902, IA: 0.8337309889253345, MAPE: 46.09033376554385%


 16%|█▌        | 4/25 [00:25<02:13,  6.35s/it]


KeyboardInterrupt: 

In [ ]:
print(f'Accuracy on test set: {get_accuracy(model2, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model2, test_loader, device=device, print_e=True)

### SNN with Synaptic layer

In [14]:
model3 = SynapticSNN(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model3)

SynapticSNN(
  (fc1): Linear(in_features=8, out_features=3500, bias=True)
  (lif): Synaptic()
  (fc2): Linear(in_features=3500, out_features=1, bias=True)
)


In [15]:
training_loop(model3, train_loader, valid_loader, device, num_epochs=25, lr=0.0005, validation=True)

  4%|▍         | 1/25 [00:20<08:16, 20.69s/it]

MAE: 8.232242779373433, MSE:126.25745839541996,
RMSE: 11.236434416460586, IA: 0.6070276439054336, MAPE: 54.66549650346819%


  8%|▊         | 2/25 [00:42<08:05, 21.11s/it]

MAE: 6.978520220276938, MSE:88.20087729500598,
RMSE: 9.391532212317966, IA: 0.805832818532277, MAPE: 47.826281028873815%


 12%|█▏        | 3/25 [01:02<07:34, 20.65s/it]

...

 96%|█████████▌| 24/25 [08:17<00:20, 20.84s/it]

MAE: 5.9485664869319494, MSE:65.26394239364875,
RMSE: 8.078610177106501, IA: 0.8792537322671139, MAPE: 39.64529049023867%


100%|██████████| 25/25 [08:37<00:00, 20.71s/it]

MAE: 5.7148208403173895, MSE:63.94562185040342,
RMSE: 7.996600643423643, IA: 0.8838679519829968, MAPE: 35.31117355672974%


Accuracy on validation dataset: 51.559999999999995%


In [16]:
print(f'Accuracy on test set: {get_accuracy(model3, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model3, test_loader, device=device, print_e=True)

Accuracy on test set: 54.620000000000005%
MAE: 5.715600851320523, MSE:68.73188526984967,
RMSE: 8.290469544594544, IA: 0.9142479890834956, MAPE: 37.17047633429125%


(5.715600851320523,
 68.73188526984967,
 8.290469544594544,
 0.9142479890834956,
 37.17047633429125)

### SNN with two Leaky layers

In [ ]:
model4 = DoubleLeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model4)

In [ ]:
training_loop(model4, train_loader, valid_loader, device, num_epochs=25, lr=0.002, validation=True)

In [ ]:
print(f'Accuracy on test set: {get_accuracy(model4, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model4, test_loader, device=device, print_e=True)

### Repeated training on random datasets

In [7]:
err_model1 = [0., 0., 0., 0., 0., 0.]
err_model2 = [0., 0., 0., 0., 0., 0.]
err_model3 = [0., 0., 0., 0., 0., 0.]
err_model4 = [0., 0., 0., 0., 0., 0.]

In [8]:
trials = 3
for i in range(trials):
    # Prepare data loaders.
    train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.75 * len(all_data)), round(0.15 * len(all_data)), round(0.1 * len(all_data))))
    train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
    test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
    valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

    # Prepare models.
    model1 = TwoLayerPerceptron(num_inputs=8, num_hidden=3500, num_outputs=1)
    model2 = LeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1)
    model3 = SynapticSNN(num_inputs=8, num_hidden=3500, num_outputs=1)
    model4 = DoubleLeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1)

    # Train models.
    print(f'------------------------------------\nTraining {i+1}')
    training_loop(model1, train_loader, valid_loader, device, num_epochs=25, lr=0.005, validation=False)
    training_loop(model2, train_loader, valid_loader, device, num_epochs=25, lr=0.0005, validation=False)
    training_loop(model3, train_loader, valid_loader, device, num_epochs=25, lr=0.0005, validation=False)
    training_loop(model4, train_loader, valid_loader, device, num_epochs=25, lr=0.0005, validation=False)

    # Gather results.
    collect_stats(model1, err_model1, test_loader, device)
    collect_stats(model2, err_model2, test_loader, device)
    collect_stats(model3, err_model3, test_loader, device)
    collect_stats(model4, err_model4, test_loader, device)

------------------------------------
Training 1


100%|██████████| 25/25 [00:11<00:00,  2.10it/s]


Accuracy on validation dataset: 52.6%


100%|██████████| 25/25 [02:11<00:00,  5.26s/it]


Accuracy on validation dataset: 51.449999999999996%


...

Accuracy on validation dataset: 53.53%


100%|██████████| 25/25 [08:33<00:00, 20.53s/it]


Accuracy on validation dataset: 53.87%


100%|██████████| 25/25 [04:07<00:00,  9.92s/it]


Accuracy on validation dataset: 53.18000000000001%


In [9]:
# Print average results
print('------Simple ANN ------')
print_average_stats(err_model1, trials)
print('------LIF SNN ------')
print_average_stats(err_model2, trials)
print('------Synaptic SNN ------')
print_average_stats(err_model3, trials)
print('------Double LIF SNN ------')
print_average_stats(err_model4, trials)

------Simple ANN ------
MAE: 6.223348741906082, MSE:77.87496346251066,
RMSE: 8.822432196005215, IA: 0.8731686590293771,
MAPE: 38.26240906891394 %, acc: 16.153333333333332 %

------LIF SNN ------
MAE: 5.9599948740642255, MSE:70.97580849449467,
RMSE: 8.424293170580057, IA: 0.8906145868669072,
MAPE: 36.826144396834216 %, acc: 17.36 %

------Synaptic SNN ------
MAE: 5.89480741050345, MSE:71.08278226297476,
RMSE: 8.430690187721938, IA: 0.891052049095577,
MAPE: 35.88559691869337 %, acc: 17.926666666666666 %

------Double LIF SNN ------
MAE: 6.093222893074224, MSE:72.02550909860128,
RMSE: 8.482812043114984, IA: 0.8892542744735851,
MAPE: 39.67798868867746 %, acc: 16.64 %

